In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
import glob
from tqdm import tqdm
import traceback
from datetime import datetime

In [ ]:
# Install required packages
print("Installing required packages...")
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install torch tqdm

Installing required packages...
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-54gzgyu3
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-54gzgyu3
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=f538e34b3482c24e33b05f1523429e36b90de8d001e8c111cb4d74462c69e3c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-is9b37rh/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/

In [ ]:
# Import whisper after installation
import whisper

In [ ]:
# Configuration
AUDIO_DIR = "preprocess/audio"
TRANSCRIPT_DIR = "preprocess/transcripts"
CSV_OUTPUT_PATH = "train/transcription_manifest.csv"
ERROR_LOG_PATH = "transcription_errors.log"

In [ ]:
# Create output directory if it doesn't exist
os.makedirs(TRANSCRIPT_DIR, exist_ok=True)

print(f"Audio directory: {AUDIO_DIR}")
print(f"Transcript directory: {TRANSCRIPT_DIR}")
print(f"CSV output: {CSV_OUTPUT_PATH}")

Audio directory: /content/drive/MyDrive/Satria_Data/train/preprocess/audio
Transcript directory: /content/drive/MyDrive/Satria_Data/train/preprocess/transcripts
CSV output: /content/drive/MyDrive/Satria_Data/train/transcription_manifest.csv


In [ ]:
# Load Whisper model (using 'large' for best accuracy)
print("Loading Whisper model...")
model = whisper.load_model("large")
print("Whisper model loaded successfully!")

Loading Whisper model...


100%|██████████████████████████████████████| 2.88G/2.88G [00:23<00:00, 134MiB/s]


Whisper model loaded successfully!


In [ ]:
def extract_id_and_emotion(filename):
    """
    Extract ID and emotion from filename like '1_Surprise.wav'
    Returns: (id, emotion) tuple
    """
    # Remove .wav extension
    name_without_ext = filename.replace('.wav', '')

    # Split by underscore
    parts = name_without_ext.split('_', 1)  # Split only on first underscore

    if len(parts) == 2:
        id_part = parts[0]
        emotion_part = parts[1]
        return id_part, emotion_part
    else:
        # Handle edge cases where filename doesn't follow expected pattern
        return name_without_ext, "Unknown"

def transcribe_audio_file(audio_path, output_json_path):
    """
    Transcribe a single audio file using Whisper and save to JSON
    Returns: True if successful, False if failed
    """
    try:
        # Transcribe with word timestamps
        result = model.transcribe(audio_path, word_timestamps=True)

        # Save the full result to JSON file
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=4, ensure_ascii=False)

        return True
    except Exception as e:
        return False

def log_error(message, error_log_path):
    """Log error message to file with timestamp"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(error_log_path, 'a', encoding='utf-8') as f:
        f.write(f"[{timestamp}] {message}\n")

In [ ]:
# Get all wav files in the audio directory
print("Scanning for audio files...")
audio_files = glob.glob(os.path.join(AUDIO_DIR, "*.wav"))
print(f"Found {len(audio_files)} wav files")

Scanning for audio files...
Found 745 wav files


In [ ]:
# Initialize results storage
results = []
successful_count = 0
failed_count = 0

In [ ]:
# Initialize error log
with open(ERROR_LOG_PATH, 'w', encoding='utf-8') as f:
    f.write(f"Transcription Error Log - Started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*80 + "\n\n")

In [ ]:
print("Starting transcription process...")
print("This may take a while for 803 files...")
# Process each audio file
for audio_path in tqdm(audio_files, desc="Transcribing audio files"):
    # Get filename
    filename = os.path.basename(audio_path)

    # Extract ID and emotion
    file_id, emotion = extract_id_and_emotion(filename)

    # Define output JSON path
    json_filename = f"{file_id}_{emotion}_transcript.json"
    json_path = os.path.join(TRANSCRIPT_DIR, json_filename)

    try:
        # Check if transcript already exists (for resuming interrupted runs)
        if os.path.exists(json_path):
            print(f"Transcript already exists for {filename}, skipping...")
            results.append({
                'id': file_id,
                'transcript_path': json_path,
                'emotion': emotion
            })
            successful_count += 1
            continue

        # Transcribe the audio file
        success = transcribe_audio_file(audio_path, json_path)

        if success:
            # Add to results
            results.append({
                'id': file_id,
                'transcript_path': json_path,
                'emotion': emotion
            })
            successful_count += 1
            print(f"✓ Successfully transcribed: {filename}")
        else:
            # Log the error
            error_msg = f"Failed to transcribe {filename} (ID: {file_id}, Emotion: {emotion})"
            log_error(error_msg, ERROR_LOG_PATH)
            failed_count += 1
            print(f"✗ Failed to transcribe: {filename}")

    except Exception as e:
        # Log unexpected errors
        error_msg = f"Unexpected error processing {filename}: {str(e)}"
        log_error(f"{error_msg}\nTraceback: {traceback.format_exc()}", ERROR_LOG_PATH)
        failed_count += 1
        print(f"✗ Error processing: {filename}")

print("\nTranscription process completed!")
print(f"Successful: {successful_count}")
print(f"Failed: {failed_count}")
print(f"Total processed: {successful_count + failed_count}")

Starting transcription process...
This may take a while for 803 files...


Transcribing audio files:   0%|          | 1/745 [00:42<8:45:45, 42.40s/it]

✓ Successfully transcribed: 1_Surprise.wav


Transcribing audio files:   0%|          | 2/745 [01:17<7:49:26, 37.91s/it]

✓ Successfully transcribed: 2_Surprise.wav


...

✓ Successfully transcribed: 801_Proud.wav


Transcribing audio files: 100%|█████████▉| 744/745 [4:21:19<00:24, 24.19s/it]

✓ Successfully transcribed: 802_Anger.wav


Transcribing audio files: 100%|██████████| 745/745 [4:21:35<00:00, 21.07s/it]

✓ Successfully transcribed: 803_Surprise.wav

Transcription process completed!
Successful: 745
Failed: 0
Total processed: 745


In [ ]:
# Create CSV manifest
if results:
    print(f"\nCreating CSV manifest with {len(results)} entries...")
    df = pd.DataFrame(results)

    # Sort by ID (convert to int for proper numeric sorting)
    try:
        df['id_numeric'] = df['id'].astype(int)
        df = df.sort_values('id_numeric').drop('id_numeric', axis=1)
    except:
        # If conversion fails, sort as strings
        df = df.sort_values('id')

    # Save to CSV
    df.to_csv(CSV_OUTPUT_PATH, index=False)
    print(f"CSV manifest saved to: {CSV_OUTPUT_PATH}")

    # Display sample of the results
    print(f"\nSample of the results:")
    print(df.head(10))

    # Display emotion distribution
    print(f"\nEmotion distribution:")
    emotion_counts = df['emotion'].value_counts()
    for emotion, count in emotion_counts.items():
        print(f"  {emotion}: {count}")

else:
    print("No successful transcriptions to save!")


Creating CSV manifest with 745 entries...
CSV manifest saved to: /content/drive/MyDrive/Satria_Data/train/transcription_manifest.csv

Sample of the results:
   id                                    transcript_path   emotion
0   1  /content/drive/MyDrive/Satria_Data/train/prepr...  Surprise
1   2  /content/drive/MyDrive/Satria_Data/train/prepr...  Surprise
2   3  /content/drive/MyDrive/Satria_Data/train/prepr...  Surprise
3   4  /content/drive/MyDrive/Satria_Data/train/prepr...  Surprise
4   5  /content/drive/MyDrive/Satria_Data/train/prepr...     Proud
5   6  /content/drive/MyDrive/Satria_Data/train/prepr...  Surprise
6   7  /content/drive/MyDrive/Satria_Data/train/prepr...     Proud
7   8  /content/drive/MyDrive/Satria_Data/train/prepr...     Proud
8   9  /content/drive/MyDrive/Satria_Data/train/prepr...     Trust
9  10  /content/drive/MyDrive/Satria_Data/train/prepr...  Surprise

Emotion distribution:
  Surprise: 304
  Trust: 173
  Proud: 147
  Joy: 49
  Anger: 36
  Fear: 16
  Sadne

In [ ]:
# Log final summary
log_error(f"Transcription completed. Success: {successful_count}, Failed: {failed_count}", ERROR_LOG_PATH)

print(f"\nError log saved to: {ERROR_LOG_PATH}")
print("Done!")


Error log saved to: /content/drive/MyDrive/Satria_Data/train/transcription_errors.log
Done!
